In [1]:
import dask.dataframe as dd
import dask_geopandas as dg

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils

import gcsfs
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
analysis_date = shared_utils.rt_dates.DATES['oct2022']

In [3]:
analysis_date

'2022-10-12'

In [20]:
bbb = 'gs://calitp-analytics-data/data-analyses/dask_test/speeds_by_operator/speeds_4_2022-10-12.parquet'

In [21]:
bbb_test = pd.read_parquet(bbb)

In [22]:
bbb_test >> head(3)

,calitp_itp_id,trip_id,route_dir_identifier,segment_sequence,min_time,max_time,min_dist,max_dist,meters_elapsed,sec_elapsed,speed_mph
0,4,1002020,2062080730,0,2022-10-12 03:57:57,2022-10-12 04:03:03,59.829328,939.252609,879.423281,306.0,6.428987
1,4,9317020,2062080730,0,2022-10-12 02:09:47,2022-10-12 02:16:18,52.782097,940.117003,887.334906,391.0,5.076645
2,4,10947020,2062080730,0,2022-10-12 04:07:31,2022-10-12 04:14:12,52.782097,960.731098,907.949001,401.0,5.065042


In [23]:
bbb_test >> filter(_.min_dist.isna()) 

,calitp_itp_id,trip_id,route_dir_identifier,segment_sequence,min_time,max_time,min_dist,max_dist,meters_elapsed,sec_elapsed,speed_mph


In [24]:
bbb_test.min_dist.isna().value_counts()

False    79574
Name: min_dist, dtype: int64

In [25]:
bbb_test.max_dist.isna().value_counts()

False    79574
Name: max_dist, dtype: int64

In [27]:
partition_speeds = 'gs://calitp-analytics-data/data-analyses/dask_test/speeds_2022-10-12'

In [29]:
ddf = dd.read_parquet(partition_speeds)